Sample script to show each models output

In [ ]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from pysb.logging import setup_logger
from pysb.simulator import CudaSSASimulator, ScipyOdeSimulator, OpenCLSSASimulator, StochKitSimulator, BngSimulator
from pysb.examples.earm_1_0 import model as earm_model
from pysb.examples.schloegl import model as schoelgl_model
from pysb.examples.kinase_cascade import model as kinase_model
from pysb.examples.michment import model as michment_model

In [ ]:
from pysb.bng import generate_equations

In [ ]:
generate_equations(kinase_model)
generate_equations(earm_model)

In [ ]:
def get_model_stats(model):
    generate_equations(model)
    return {
        'model_name': model.name,
        'n_species': len(model.species),
        'n_reactions': len(model.reactions),
        'n_parameters': len(model.parameters)
        
    }

In [ ]:
get_model_stats(kinase_model)

In [ ]:
get_model_stats(earm_model)

In [ ]:
get_model_stats(michment_model)

In [ ]:
get_model_stats(schoelgl_model)

In [ ]:
tspan = np.linspace(0, 10, 101)
sim = OpenCLSSASimulator(michment_model, tspan=tspan, verbose=True)
traj = sim.run(number_sim=100).dataframe
traj.reset_index(inplace=True)

# ode = ScipyOdeSimulator(michment_model, tspan=tspan).run()

In [ ]:
print(traj.head(10))
obs = ['E_free','S_free', 'ES_complex', 'Product']

In [ ]:
def run_model(model, tspan, n_sim=1000, obs=None):
    sim = OpenCLSSASimulator(model, tspan=tspan, verbose=True, precision=np.float64)

    ode = ScipyOdeSimulator(model, tspan=tspan).run()

#     sim = CUDASimulator(model, tspan=tspan, verbose=True, )
    traj = sim.run(tspan=tspan, number_sim=n_sim)
    plot(traj, ode, tspan, obs, obs)
    return
    for n, i in enumerate(model.species):
        name = '__s{}'.format(n)
        print(name)
        plot(traj, ode, tspan, name, i)

def plot(traj, ode, tspan, obs, title):
    x = traj.dataframe[obs].unstack(0).values

    plt.figure()
    plt.title(title)
    # create line traces
    plt.plot(tspan, x, '0.5', lw=2, alpha=0.25)  # individual trajectories
    plt.plot(tspan, x.mean(1), 'k-*', lw=3, label="Mean")
    plt.plot(tspan, x.min(1), 'b--', lw=3, label="Minimum")
    plt.plot(tspan, x.max(1), 'r--', lw=3, label="Maximum")

    # adding ODE solution to plot
    plt.plot(tspan, ode.dataframe[name], 'g--', lw=3, label="ODE")

    plt.xlabel('Time(s)')
    plt.ylabel('Number of molecules')
    if not os.path.exists("figures"):
        os.mkdir('figures')
    plt.savefig('figures/{}.png'.format(obs), bbox_inches='tight')
    

In [ ]:

tspan = np.linspace(0, 8000, 101)
name = 'tBid_total'
# name = 'tBid_total'
run_model(earm_model, tspan, n_sim=1000, obs=name)

In [ ]:
tspan = np.linspace(0, 300, 101)
name = 'ppERK'
run_model(kinase_model, tspan, 100, obs=name)
    

In [ ]:
tspan = np.linspace(0, 100, 101)
name = 'X_total'

run_model(scholgl_model, tspan, n_sim=2**12, obs=name)

In [ ]:

def save_output(model, tspan, n_sim=1000):
    name = model.name.split('.')[-1]
    
    def save_traj(traj, sim):
        s_name = "{name}_{n_sim}_{sim}.csv.gz".format(name=name, n_sim=n_sim, sim=sim)
        traj.to_csv(s_name, compression='gzip')
        
    print("Running OpenCLSimulator")
    traj = OpenCLSimulator(model, platform='NVIDIA',device='gpu'
                          ).run(tspan=tspan, number_sim=n_sim).dataframe
    save_traj(traj, 'opencl')
    
    print("Running CUDASimulator")
    traj = CUDASimulator(model).run(tspan=tspan, number_sim=n_sim).dataframe
    save_traj(traj, 'cuda')
    
    print("Running BngSimulator")
    traj = BngSimulator(model).run(tspan=tspan, n_runs=n_sim).dataframe
    save_traj(traj, 'bng')
    
    print("Running StochKitSimulator")
    traj = StochKitSimulator(model).run(tspan=tspan, n_runs=n_sim).dataframe
    save_traj(traj, 'stochkit')
    
def load_traj(model, n_sim=1000):
    
    name = model.name.split('.')[-1]
    bng_file = "{name}_{n_sim}_{sim}.csv.gz".format(name=name, n_sim=n_sim, sim='bng')
    stochkit_file = "{name}_{n_sim}_{sim}.csv.gz".format(name=name, n_sim=n_sim, sim='stochkit')
    cuda_file = "{name}_{n_sim}_{sim}.csv.gz".format(name=name, n_sim=n_sim, sim='cuda')
    opencl_file = "{name}_{n_sim}_{sim}.csv.gz".format(name=name, n_sim=n_sim, sim='opencl')
    
    bng_data = pd.read_csv(bng_file)
    stochkit_data = pd.read_csv(stochkit_file)
    cuda_data = pd.read_csv(cuda_file)
    opencl_data = pd.read_csv(opencl_file)
    bng_data['simulator'] = 'bng'
    stochkit_data['simulator'] = 'stochkit'
    cuda_data['simulator'] = 'CUDASimulator'
    opencl_data['simulator'] = 'OpenCLSimulator'

    df = pd.concat([bng_data, stochkit_data, cuda_data, opencl_data], ignore_index=True)
    return df

In [ ]:
def create_plot(df, obs):
    plt.figure(figsize=(6,6))
    ax = sns.lineplot(
        x="time", y=obs,
        markers=True, dashes=True,
        ci='sd', 
        estimator=np.average,
        hue="simulator", 
        data=df
    )

In [ ]:
tspan = np.linspace(0, 100, 301)
scholgl_model.parameters['X_0'].value = 250
save_output(scholgl_model, tspan, n_sim=1000)